In [10]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer #tokenizes and counts words
from sklearn.feature_extraction.text import TfidfVectorizer #alternatively, use TfidfTransformer()
from sklearn import metrics
import requests
from sklearn.feature_selection import RFE, chi2
from scipy.spatial import distance
from pandas.core.frame import DataFrame
from nltk import FreqDist
from sklearn.metrics.pairwise import cosine_similarity
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
import string
from scipy.stats import entropy
from numpy.linalg import norm
from scipy.spatial import distance
import spacy
from collections import Counter
import math
from math import log

In [11]:
from spacy.lang.en import English
from spacy.lang.es import Spanish
from spacy.lang.el import Greek
from spacy.lang.zh import Chinese

In [28]:
def dice(im1, im2):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / (im1.sum() + im2.sum())                                                                          

In [3]:
im1 = [3,4,0,5,3]
im2 = [0,3,4,5,2]
im1 = np.asarray(im1).astype(np.bool)
im2 = np.asarray(im2).astype(np.bool)
intersection = np.logical_and(im1, im2)
print(intersection)

[False  True False  True  True]


array([ True,  True, False,  True,  True])

In [12]:
df = pd.read_csv('NA28_11_09_11PM.csv', encoding = 'utf-16', sep = '\t')
df2 = pd.read_csv('SBL_11_09_11PM.csv', encoding = 'utf-16', sep = '\t')
df3 = pd.read_csv('Tyndale_11_09_11PM.csv', encoding = 'utf-16', sep = '\t')

In [13]:
def cosine_text(text):
    text_list = []
    for t in text['Verse']:
        text_list.append(t)

    text_str = ' '.join(text_list)
    return text_str

In [14]:
def word_list(text):
    spacy_stopwords = spacy.lang.el.stop_words.STOP_WORDS
    # spacy_stopwords is a hardcoded set
    nlp = Greek()
    tokenizer = nlp.Defaults.create_tokenizer(nlp)
    tokens = tokenizer(text)
    tokens1 = tokenizer(hbr)
    #token_list = []
    lemma_list = []
    lemma_list1 = []
    for token in tokens:
        if token.lemma_.lower() not in spacy_stopwords:
        #token_list.append(token.text)
            lemma_list.append(token.lemma_)
    for token in tokens1:
        if token.lemma_.lower() not in spacy_stopwords:
        #token_list.append(token.text)
            lemma_list1.append(token.lemma_)
    #token_list
    dict1 = {}
    dict2 = {}
    for key in lemma_list:
        dict1[key] = dict1.get(key, 0) + 1
    for key in lemma_list1:
        dict2[key] = dict2.get(key, 0) + 1
    l1 = []
    l2 = []
    for word in dict1:
        if word not in dict2:
            l2.append(word)
    for word in dict2:
        if word not in dict1:
            l1.append(word)
    for x in l1:
        dict1[x] = dict1.get(x,0)
    for y in l2:
        dict2[y] = dict2.get(y,0)
    s_dict1 = sorted(dict1.items())
    s_dict2 = sorted(dict2.items())
    m1 = []
    m2 = []
    for m in s_dict1:
        m1.append(m[1])
    for n in s_dict2:
        m2.append(n[1])
    return [m1,m2]

In [21]:
def word_list1(text):
    spacy_stopwords = spacy.lang.el.stop_words.STOP_WORDS
    # spacy_stopwords is a hardcoded set
    nlp = Greek()
    tokenizer = nlp.Defaults.create_tokenizer(nlp)
    tokens = tokenizer(text)
    tokens1 = tokenizer(hbr)
    #token_list = []
    lemma_list = []
    lemma_list1 = []
    for token in tokens:
        if token.lemma_.lower() not in spacy_stopwords:
        #token_list.append(token.text)
            lemma_list.append(token.lemma_)
    for token in tokens1:
        if token.lemma_.lower() not in spacy_stopwords:
        #token_list.append(token.text)
            lemma_list1.append(token.lemma_)
    #token_list
    dict1 = {}
    dict2 = {}
    for key in lemma_list:
        dict1[key] = dict1.get(key, 0) + 1
    for key in lemma_list1:
        dict2[key] = dict2.get(key, 0) + 1
    l1 = []
    l2 = []
    for word in dict1:
        if word not in dict2:
            l2.append(word)
    for word in dict2:
        if word not in dict1:
            l1.append(word)
    for x in l1:
        dict1[x] = dict1.get(x,0)
    for y in l2:
        dict2[y] = dict2.get(y,0)
    s_dict1 = sorted(dict1.items())
    s_dict2 = sorted(dict2.items())
    m1 = []
    m2 = []
    for m in s_dict1:
        m1.append(m[1])
    for n in s_dict2:
        m2.append(n[1])
    return [m1,m2]

## NA28_Version_Analytics

In [22]:
df.BookName.unique()

array(['Mt', 'Mk', 'Lk', 'Joh', 'Apg', 'Röm', '1.Kor', '2.Kor', 'Gal',
       'Eph', 'Phil', 'Kol', '1.Thess', '2.Thess', '1.Tim', '2.Tim',
       'Tit', 'Phlm', 'heb', 'Jak', '1.Petr', '2.Petr', '1.Joh', '2.Joh',
       '3.Joh', 'Jud', 'Offb'], dtype=object)

In [23]:
Matthew = df[df.BookName == 'Mt']
Matthew['Author'] = 'Matthew'
Mark = df[df.BookName == 'Mk']
Mark['Author'] = 'Mark'
Luke = df[df.BookName == 'Lk']
Luke['Author'] = 'Luke'
John = df[df.BookName == 'Joh']
John['Author'] = 'John'
Acts = df[df.BookName == 'Apg']
Acts['Author'] = 'Luke'
Luke_Book = pd.concat([Luke,Acts],axis = 0)
James = df[df.BookName == 'Jak']
James['Author'] = 'James'
f_Peter = df[df.BookName == '1.Petr']
s_Peter = df[df.BookName == '2.Petr']
Peter_letter = pd.concat([f_Peter, s_Peter], axis = 0)
Peter_letter['Author'] = 'Peter'
f_John = df[df.BookName == '1.Joh']
s_John = df[df.BookName == '2.Joh']
t_John = df[df.BookName == '3.Joh']
John_letter = pd.concat([f_John, s_John, t_John], axis = 0)
John_letter['Author'] = 'John'
Jude = df[df.BookName == 'Jud']
Jude['Author'] = 'Jude'
Rev = df[df.BookName == 'Offb']
Rev['Author'] = 'John'
John_Book = pd.concat([John,John_letter,Rev],axis = 0)

In [24]:
Romans = df[df.BookName == 'Röm']
Romans['Author'] = 'Paul'
f_cor = df[df.BookName == '1.Kor']
f_cor['Author'] = 'Paul'
s_cor = df[df.BookName == '2.Kor']
s_cor['Author'] = 'Paul'
Gal = df[df.BookName == 'Gal']
Gal['Author'] = 'Paul'
Eph = df[df.BookName == 'Eph']
Eph['Author'] = 'Paul'
Phi = df[df.BookName == 'Phil']
Phi['Author'] = 'Paul'
Col = df[df.BookName == 'Kol']
Col['Author'] = 'Paul'
f_the = df[df.BookName == '1.Thess']
f_the['Author'] = 'Paul'
s_the = df[df.BookName == '2.Thess']
s_the['Author'] = 'Paul'
f_tim = df[df.BookName == '1.Tim']
f_tim['Author'] = 'Paul'
s_tim = df[df.BookName == '2.Tim']
s_tim['Author'] = 'Paul'
Titus = df[df.BookName == 'Tit']
Titus['Author'] = 'Paul'
Philemon = df[df.BookName == 'Phlm']
Philemon['Author'] = 'Paul'
Paul_letters = pd.concat([Romans,f_cor,s_cor,Gal,Eph,Phi,Col,f_the,s_the,f_tim,s_tim,Titus,Philemon],axis = 0)
Heb = df[df.BookName == 'heb']
Heb['Author'] = 'Paul?'

In [25]:
g1 = cosine_text(Matthew)
g2 = cosine_text(Mark)
g3 = cosine_text(Luke)
g4 = cosine_text(John)
acts = cosine_text(Acts)
rms = cosine_text(Romans)
fcor = cosine_text(f_cor)
scor = cosine_text(s_cor)
gal = cosine_text(Gal)
eph = cosine_text(Eph)
phi = cosine_text(Phi)
col = cosine_text(Col)
fthe = cosine_text(f_the)
sthe = cosine_text(s_the)
ftim = cosine_text(f_tim)
stim = cosine_text(s_tim)
titus = cosine_text(Titus)
philemon = cosine_text(Philemon)
jams = cosine_text(James)
fpet = cosine_text(f_Peter)
spet = cosine_text(s_Peter)
fjohn = cosine_text(f_John)
sjohn = cosine_text(s_John)
tjohn = cosine_text(t_John.dropna())
jud = cosine_text(Jude)
rev = cosine_text(Rev)
hbr = cosine_text(Heb)

In [26]:
g11 = word_list(g1)
g21 = word_list(g2)
g31 = word_list(g3)
g41 = word_list(g4)
acts1 = word_list(acts)
rms1 = word_list(rms)
fcor1 = word_list(fcor)
scor1 = word_list(scor)
gal1 = word_list(gal)
eph1 = word_list(eph)
phi1 = word_list(phi)
col1 = word_list(col)
fthe1 = word_list(fthe)
sthe1 = word_list(sthe)
ftim1 = word_list(ftim)
stim1 = word_list(stim)
titus1 = word_list(titus)
philemon1 = word_list(philemon)
jams1 = word_list(jams)
fpet1 = word_list(fpet)
spet1 = word_list(spet)
fjohn1 = word_list(fjohn)
sjohn1 = word_list(sjohn)
tjohn1 = word_list(tjohn)
jud1 = word_list(jud)
rev1 = word_list(rev)
hbr1 = word_list(hbr)

In [29]:
documents = [g11,g21,g31,g41,acts1,rms1,fcor1,scor1,gal1,eph1,phi1,col1,fthe1,sthe1,ftim1,stim1,titus1,philemon1,jams1,fpet1,spet1,fjohn1,sjohn1,tjohn1,jud1,rev1]
s_dice = []
s_hel = []
s_jac = []
s_jen = []
a = []
for x in range(0,len(documents)):
    a.append(distance.dice(documents[x][0],documents[x][1]))
    s_dice.append(dice(documents[x][0],documents[x][1]))
    s_hel.append(hellinger(documents[x][0],documents[x][1]))
    s_jac.append(jaccard(documents[x][0],documents[x][1]))
    s_jen.append(distance.jensenshannon(documents[x][0],documents[x][1]))


[0.19900826446280992,
 0.20012404382881951,
 0.1875,
 0.20788979336255478,
 0.1865581675708258,
 0.23455166973442018,
 0.21860701576004068,
 0.2206060606060606,
 0.20620842572062084,
 0.2030608435983576,
 0.18692338547934215,
 0.1772875816993464,
 0.18440905280804695,
 0.16315307057745188,
 0.17484662576687116,
 0.16892168921689216,
 0.11360513055428309,
 0.09566968781470292,
 0.1831888428194497,
 0.18737749901999215,
 0.17848206839032527,
 0.18119364534134821,
 0.08251676121712223,
 0.07632800412583807,
 0.12182254196642686,
 0.21287379624936645]

In [14]:
nlp = spacy.load('el_core_news_md')

In [15]:
s_cosine = []
documents = [g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,27))
doc1 = nlp(hbr)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine.append(similarity)

In [16]:
n_cosine = []
documents = [hbr,g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,27))
for x in range(1,len(documents)):
    two_books = [hbr,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m5 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Hebrews','Compared Book'])
    # df_m5.to_csv(str(bindex[x])+'compared Hebrews term weight matrix.csv', encoding = 'utf-16',sep = '\t', index=False)
    cos = tuple(cosine_similarity(df_m5)[0])[1]
    n_cosine.append(cos)

In [17]:
# dice, js divergence, hellinger distance
n_dice = []
n_hel = []
n_jac = []
n_jen = []
documents = [hbr,g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,len(documents)))
for x in range(1,len(documents)):
    two_books = [hbr,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice.append(dice(m1[0],m2[0]))
    n_hel.append(hellinger(m1[0],m2[0]))
    n_jac.append(jaccard(m1[0],m2[0]))
    n_jen.append(distance.jensenshannon(m1[0],m2[0]))
    

In [18]:
NA28 = {'SpacyCosine':s_cosine, 'SpacyDice':s_dice, 'SpacyHellinger':s_hel, 'SpacyJaccard':s_jac, 'SpacyJSdivergence':s_jen, 'NLTKCosine':n_cosine, 'NLTKDice':n_dice, 'NLTKHellinger':n_hel, 'NLTKJaccard':n_jac, 'NLTKJSdivergence':n_jen}
NA28 = pd.DataFrame(NA28)

In [19]:
mat_s1 = cosine_text(Matthew)
mark_s1 = cosine_text(Mark)
luke_s1 = cosine_text(Luke_Book)
jam_s1 = cosine_text(James)
pet_s1 = cosine_text(Peter_letter)
john_s1 = cosine_text(John_Book.dropna())
jude_s1 = cosine_text(Jude)
pl_s1 = cosine_text(Paul_letters)

In [20]:
mat_s2 = word_list(mat_s1)
mark_s2 = word_list(mark_s1)
luke_s2 = word_list(luke_s1)
jam_s2 = word_list(jam_s1)
pet_s2 = word_list(pet_s1)
john_s2 = word_list(john_s1)
jude_s2 = word_list(jude_s1)
pl_s2 = word_list(pl_s1)

In [21]:
s_dice1 = []
s_hel1 = []
s_jac1 = []
s_jen1 = []
documents = [pl_s2,mat_s2,mark_s2,luke_s2,jam_s2,pet_s2,john_s2,jude_s2]
for x in range(0,len(documents)):
    s_dice1.append(dice(documents[x][0],documents[x][1]))
    s_hel1.append(hellinger(documents[x][0],documents[x][1]))
    s_jac1.append(jaccard(documents[x][0],documents[x][1]))
    s_jen1.append(distance.jensenshannon(documents[x][0],documents[x][1]))

In [22]:
s_cosine1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
doc1 = nlp(hbr)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine1.append(similarity)

In [23]:
n_cosine1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [hbr,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m6 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Paul','compared books'])
    # df_m6.to_csv(str(authorindex[x])+'author compare with Paul term weight matrix.csv', encoding = 'utf-8', index=False)
    cos = tuple(cosine_similarity(df_m6)[0])[1]
    n_cosine1.append(cos)

In [24]:
n_dice1 = []
n_hel1 = []
n_jac1 = []
n_jen1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [hbr,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice1.append(dice(m1[0],m2[0]))
    n_hel1.append(hellinger(m1[0],m2[0]))
    n_jac1.append(jaccard(m1[0],m2[0]))
    n_jen1.append(distance.jensenshannon(m1[0],m2[0]))

In [25]:
NA281 = {'SpacyCosine':s_cosine1, 'SpacyDice':s_dice1, 'SpacyHellinger':s_hel1, 'SpacyJaccard':s_jac1, 'SpacyJSdivergence':s_jen1, 'NLTKCosine':n_cosine1, 'NLTKDice':n_dice1, 'NLTKHellinger':n_hel1, 'NLTKJaccard':n_jac1, 'NLTKJSdivergence':n_jen1}
NA281 = pd.DataFrame(NA281)

In [26]:
mat_s4 = word_list1(mat_s1)
mark_s4 = word_list1(mark_s1)
luke_s4 = word_list1(luke_s1)
jam_s4 = word_list1(jam_s1)
pet_s4 = word_list1(pet_s1)
john_s4 = word_list1(john_s1)
jude_s4 = word_list1(jude_s1)

In [27]:
s_dice2 = []
s_hel2 = []
s_jac2 = []
s_jen2 = []
documents = [mat_s4,mark_s4,luke_s4,jam_s4,pet_s4,john_s4,jude_s4]
for x in range(0,len(documents)):
    s_dice2.append(dice(documents[x][0],documents[x][1]))
    s_hel2.append(hellinger(documents[x][0],documents[x][1]))
    s_jac2.append(jaccard(documents[x][0],documents[x][1]))
    s_jen2.append(distance.jensenshannon(documents[x][0],documents[x][1]))

In [28]:
s_cosine2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
doc1 = nlp(pl_s1)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine2.append(similarity)

In [29]:
n_cosine2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [pl_s1,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m7 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Paul','compared books'])
    # df_m6.to_csv(str(authorindex[x])+'author compare with Paul term weight matrix.csv', encoding = 'utf-8', index=False)
    cos = tuple(cosine_similarity(df_m7)[0])[1]
    n_cosine2.append(cos)

In [30]:
n_dice2 = []
n_hel2 = []
n_jac2 = []
n_jen2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [pl_s1,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice2.append(dice(m1[0],m2[0]))
    n_hel2.append(hellinger(m1[0],m2[0]))
    n_jac2.append(jaccard(m1[0],m2[0]))
    n_jen2.append(distance.jensenshannon(m1[0],m2[0]))

In [31]:
NA282 = {'SpacyCosine':s_cosine2, 'SpacyDice':s_dice2, 'SpacyHellinger':s_hel2, 'SpacyJaccard':s_jac2, 'SpacyJSdivergence':s_jen2, 'NLTKCosine':n_cosine2, 'NLTKDice':n_dice2, 'NLTKHellinger':n_hel2, 'NLTKJaccard':n_jac2, 'NLTKJSdivergence':n_jen2}
NA282 = pd.DataFrame(NA282)

In [32]:
NA28_all = pd.concat([NA28, NA281, NA282], axis = 0)

## SBL_Version_Analytics

In [33]:
df2.BookName.unique()

array(['Mt', 'Mk', 'Lk', 'Jn', 'Ac', 'Ro', '1Co', '2Co', 'Ga', 'Eph',
       'Php', 'Col', '1Th', '2Th', '1Ti', '2Ti', 'Tit', 'Phm', 'Heb',
       'Jas', '1Pe', '2Pe', '1Jn', '2Jn', '3Jn', 'Jud', 'Re'],
      dtype=object)

In [34]:
Matthew = df2[df2.BookName == 'Mt']
Matthew['Author'] = 'Matthew'
Mark = df2[df2.BookName == 'Mk']
Mark['Author'] = 'Mark'
Luke = df2[df2.BookName == 'Lk']
Luke['Author'] = 'Luke'
John = df2[df2.BookName == 'Jn']
John['Author'] = 'John'
Acts = df2[df2.BookName == 'Ac']
Acts['Author'] = 'Luke'
Luke_Book = pd.concat([Luke,Acts],axis = 0)
James = df2[df2.BookName == 'Jas']
James['Author'] = 'James'
f_Peter = df2[df2.BookName == '1Pe']
s_Peter = df2[df2.BookName == '2Pe']
Peter_letter = pd.concat([f_Peter, s_Peter], axis = 0)
Peter_letter['Author'] = 'Peter'
f_John = df2[df2.BookName == '1Jn']
s_John = df2[df2.BookName == '2Jn']
t_John = df2[df2.BookName == '3Jn']
John_letter = pd.concat([f_John, s_John, t_John], axis = 0)
John_letter['Author'] = 'John'
Jude = df2[df2.BookName == 'Jud']
Jude['Author'] = 'Jude'
Rev = df2[df2.BookName == 'Re']
Rev['Author'] = 'John'
John_Book = pd.concat([John,John_letter,Rev],axis = 0)

In [35]:
Romans = df2[df2.BookName == 'Ro']
Romans['Author'] = 'Paul'
f_cor = df2[df2.BookName == '1Co']
f_cor['Author'] = 'Paul'
s_cor = df2[df2.BookName == '2Co']
s_cor['Author'] = 'Paul'
Gal = df2[df2.BookName == 'Ga']
Gal['Author'] = 'Paul'
Eph = df2[df2.BookName == 'Eph']
Eph['Author'] = 'Paul'
Phi = df2[df2.BookName == 'Php']
Phi['Author'] = 'Paul'
Col = df2[df2.BookName == 'Col']
Col['Author'] = 'Paul'
f_the = df2[df2.BookName == '1Th']
f_the['Author'] = 'Paul'
s_the = df2[df2.BookName == '2Th']
s_the['Author'] = 'Paul'
f_tim = df2[df2.BookName == '1Ti']
f_tim['Author'] = 'Paul'
s_tim = df2[df2.BookName == '2Ti']
s_tim['Author'] = 'Paul'
Titus = df2[df2.BookName == 'Tit']
Titus['Author'] = 'Paul'
Philemon = df2[df2.BookName == 'Phm']
Philemon['Author'] = 'Paul'
Paul_letters = pd.concat([Romans,f_cor,s_cor,Gal,Eph,Phi,Col,f_the,s_the,f_tim,s_tim,Titus,Philemon],axis = 0)
Heb = df2[df2.BookName == 'Heb']
Heb['Author'] = 'Paul?'

In [36]:
g1 = cosine_text(Matthew)
g2 = cosine_text(Mark)
g3 = cosine_text(Luke)
g4 = cosine_text(John)
acts = cosine_text(Acts)
rms = cosine_text(Romans)
fcor = cosine_text(f_cor)
scor = cosine_text(s_cor)
gal = cosine_text(Gal)
eph = cosine_text(Eph)
phi = cosine_text(Phi)
col = cosine_text(Col)
fthe = cosine_text(f_the)
sthe = cosine_text(s_the)
ftim = cosine_text(f_tim)
stim = cosine_text(s_tim)
titus = cosine_text(Titus)
philemon = cosine_text(Philemon)
jams = cosine_text(James)
fpet = cosine_text(f_Peter)
spet = cosine_text(s_Peter)
fjohn = cosine_text(f_John)
sjohn = cosine_text(s_John)
tjohn = cosine_text(t_John.dropna())
jud = cosine_text(Jude)
rev = cosine_text(Rev)
hbr = cosine_text(Heb)

In [37]:
g11 = word_list(g1)
g21 = word_list(g2)
g31 = word_list(g3)
g41 = word_list(g4)
acts1 = word_list(acts)
rms1 = word_list(rms)
fcor1 = word_list(fcor)
scor1 = word_list(scor)
gal1 = word_list(gal)
eph1 = word_list(eph)
phi1 = word_list(phi)
col1 = word_list(col)
fthe1 = word_list(fthe)
sthe1 = word_list(sthe)
ftim1 = word_list(ftim)
stim1 = word_list(stim)
titus1 = word_list(titus)
philemon1 = word_list(philemon)
jams1 = word_list(jams)
fpet1 = word_list(fpet)
spet1 = word_list(spet)
fjohn1 = word_list(fjohn)
sjohn1 = word_list(sjohn)
tjohn1 = word_list(tjohn)
jud1 = word_list(jud)
rev1 = word_list(rev)
hbr1 = word_list(hbr)

In [38]:
documents = [g11,g21,g31,g41,acts1,rms1,fcor1,scor1,gal1,eph1,phi1,col1,fthe1,sthe1,ftim1,stim1,titus1,philemon1,jams1,fpet1,spet1,fjohn1,sjohn1,tjohn1,jud1,rev1]
s_dice = []
s_hel = []
s_jac = []
s_jen = []
for x in range(0,len(documents)):
    
    s_dice.append(dice(documents[x][0],documents[x][1]))
    s_hel.append(hellinger(documents[x][0],documents[x][1]))
    s_jac.append(jaccard(documents[x][0],documents[x][1]))
    s_jen.append(distance.jensenshannon(documents[x][0],documents[x][1]))

In [39]:
nlp = spacy.load('el_core_news_md')

In [40]:
s_cosine = []
documents = [g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,27))
doc1 = nlp(hbr)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine.append(similarity)

In [41]:
n_cosine = []
documents = [hbr,g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,27))
for x in range(1,len(documents)):
    two_books = [hbr,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m5 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Hebrews','Compared Book'])
    # df_m5.to_csv(str(bindex[x])+'compared Hebrews term weight matrix.csv', encoding = 'utf-16',sep = '\t', index=False)
    cos = tuple(cosine_similarity(df_m5)[0])[1]
    n_cosine.append(cos)

In [42]:
# dice, js divergence, hellinger distance
n_dice = []
n_hel = []
n_jac = []
n_jen = []
documents = [hbr,g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,len(documents)))
for x in range(1,len(documents)):
    two_books = [hbr,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice.append(dice(m1[0],m2[0]))
    n_hel.append(hellinger(m1[0],m2[0]))
    n_jac.append(jaccard(m1[0],m2[0]))
    n_jen.append(distance.jensenshannon(m1[0],m2[0]))
    

In [43]:
SBL = {'SpacyCosine':s_cosine, 'SpacyDice':s_dice, 'SpacyHellinger':s_hel, 'SpacyJaccard':s_jac, 'SpacyJSdivergence':s_jen, 'NLTKCosine':n_cosine, 'NLTKDice':n_dice, 'NLTKHellinger':n_hel, 'NLTKJaccard':n_jac, 'NLTKJSdivergence':n_jen}
SBL = pd.DataFrame(SBL)

In [44]:
mat_s1 = cosine_text(Matthew)
mark_s1 = cosine_text(Mark)
luke_s1 = cosine_text(Luke_Book)
jam_s1 = cosine_text(James)
pet_s1 = cosine_text(Peter_letter)
john_s1 = cosine_text(John_Book.dropna())
jude_s1 = cosine_text(Jude)
pl_s1 = cosine_text(Paul_letters)

In [45]:
mat_s2 = word_list(mat_s1)
mark_s2 = word_list(mark_s1)
luke_s2 = word_list(luke_s1)
jam_s2 = word_list(jam_s1)
pet_s2 = word_list(pet_s1)
john_s2 = word_list(john_s1)
jude_s2 = word_list(jude_s1)
pl_s2 = word_list(pl_s1)

In [46]:
s_dice1 = []
s_hel1 = []
s_jac1 = []
s_jen1 = []
documents = [pl_s2,mat_s2,mark_s2,luke_s2,jam_s2,pet_s2,john_s2,jude_s2]
for x in range(0,len(documents)):
    s_dice1.append(dice(documents[x][0],documents[x][1]))
    s_hel1.append(hellinger(documents[x][0],documents[x][1]))
    s_jac1.append(jaccard(documents[x][0],documents[x][1]))
    s_jen1.append(distance.jensenshannon(documents[x][0],documents[x][1]))

In [47]:
s_cosine1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
doc1 = nlp(hbr)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine1.append(similarity)

In [48]:
n_cosine1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [hbr,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m6 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Paul','compared books'])
    # df_m6.to_csv(str(authorindex[x])+'author compare with Paul term weight matrix.csv', encoding = 'utf-8', index=False)
    cos = tuple(cosine_similarity(df_m6)[0])[1]
    n_cosine1.append(cos)

In [49]:
n_dice1 = []
n_hel1 = []
n_jac1 = []
n_jen1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [hbr,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice1.append(dice(m1[0],m2[0]))
    n_hel1.append(hellinger(m1[0],m2[0]))
    n_jac1.append(jaccard(m1[0],m2[0]))
    n_jen1.append(distance.jensenshannon(m1[0],m2[0]))

In [50]:
SBL1 = {'SpacyCosine':s_cosine1, 'SpacyDice':s_dice1, 'SpacyHellinger':s_hel1, 'SpacyJaccard':s_jac1, 'SpacyJSdivergence':s_jen1, 'NLTKCosine':n_cosine1, 'NLTKDice':n_dice1, 'NLTKHellinger':n_hel1, 'NLTKJaccard':n_jac1, 'NLTKJSdivergence':n_jen1}
SBL1 = pd.DataFrame(SBL1)

In [51]:
mat_s4 = word_list1(mat_s1)
mark_s4 = word_list1(mark_s1)
luke_s4 = word_list1(luke_s1)
jam_s4 = word_list1(jam_s1)
pet_s4 = word_list1(pet_s1)
john_s4 = word_list1(john_s1)
jude_s4 = word_list1(jude_s1)

In [52]:
s_dice2 = []
s_hel2 = []
s_jac2 = []
s_jen2 = []
documents = [mat_s4,mark_s4,luke_s4,jam_s4,pet_s4,john_s4,jude_s4]
for x in range(0,len(documents)):
    s_dice2.append(dice(documents[x][0],documents[x][1]))
    s_hel2.append(hellinger(documents[x][0],documents[x][1]))
    s_jac2.append(jaccard(documents[x][0],documents[x][1]))
    s_jen2.append(distance.jensenshannon(documents[x][0],documents[x][1]))

In [53]:
s_cosine2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
doc1 = nlp(pl_s1)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine2.append(similarity)

In [54]:
n_cosine2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [pl_s1,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m7 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Paul','compared books'])
    # df_m6.to_csv(str(authorindex[x])+'author compare with Paul term weight matrix.csv', encoding = 'utf-8', index=False)
    cos = tuple(cosine_similarity(df_m7)[0])[1]
    n_cosine2.append(cos)

In [55]:
n_dice2 = []
n_hel2 = []
n_jac2 = []
n_jen2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [pl_s1,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice2.append(dice(m1[0],m2[0]))
    n_hel2.append(hellinger(m1[0],m2[0]))
    n_jac2.append(jaccard(m1[0],m2[0]))
    n_jen2.append(distance.jensenshannon(m1[0],m2[0]))

In [56]:
SBL2 = {'SpacyCosine':s_cosine2, 'SpacyDice':s_dice2, 'SpacyHellinger':s_hel2, 'SpacyJaccard':s_jac2, 'SpacyJSdivergence':s_jen2, 'NLTKCosine':n_cosine2, 'NLTKDice':n_dice2, 'NLTKHellinger':n_hel2, 'NLTKJaccard':n_jac2, 'NLTKJSdivergence':n_jen2}
SBL2 = pd.DataFrame(SBL2)

In [57]:
SBL_all = pd.concat([SBL, SBL1, SBL2], axis = 0)

## Tyndale_Version_Analytics

In [58]:
df3.BookName.unique()

array(['Matthew', 'Mark', 'Luke', 'John', 'Acts', 'Romans',
       '1Corinthians', '2Corinthians', 'Galatians', 'Ephesians',
       'Philippians', 'Colossians', '1Thessalonians', '2Thessalonians',
       '1Timothy', '2Timothy', 'Titus', 'Philemon', 'Hebrews', 'James',
       '1Peter', '2Peter', '1John', '2John', '3John', 'Jude',
       'Revelation'], dtype=object)

In [59]:
Matthew = df3[df3.BookName == 'Matthew']
Matthew['Author'] = 'Matthew'
Mark = df3[df3.BookName == 'Mark']
Mark['Author'] = 'Mark'
Luke = df3[df3.BookName == 'Luke']
Luke['Author'] = 'Luke'
John = df3[df3.BookName == 'John']
John['Author'] = 'John'
Acts = df3[df3.BookName == 'Acts']
Acts['Author'] = 'Luke'
Luke_Book = pd.concat([Luke,Acts],axis = 0)
James = df3[df3.BookName == 'James']
James['Author'] = 'James'
f_Peter = df3[df3.BookName == '1Peter']
s_Peter = df3[df3.BookName == '2Peter']
Peter_letter = pd.concat([f_Peter, s_Peter], axis = 0)
Peter_letter['Author'] = 'Peter'
f_John = df3[df3.BookName == '1John']
s_John = df3[df3.BookName == '2John']
t_John = df3[df3.BookName == '3John']
John_letter = pd.concat([f_John, s_John, t_John], axis = 0)
John_letter['Author'] = 'John'
Jude = df3[df3.BookName == 'Jude']
Jude['Author'] = 'Jude'
Rev = df3[df3.BookName == 'Revelation']
Rev['Author'] = 'John'
John_Book = pd.concat([John,John_letter,Rev],axis = 0)

In [60]:
Romans = df3[df3.BookName == 'Romans']
Romans['Author'] = 'Paul'
f_cor = df3[df3.BookName == '1Corinthians']
f_cor['Author'] = 'Paul'
s_cor = df3[df3.BookName == '2Corinthians']
s_cor['Author'] = 'Paul'
Gal = df3[df3.BookName == 'Galatians']
Gal['Author'] = 'Paul'
Eph = df3[df3.BookName == 'Ephesians']
Eph['Author'] = 'Paul'
Phi = df3[df3.BookName == 'Philippians']
Phi['Author'] = 'Paul'
Col = df3[df3.BookName == 'Colossians']
Col['Author'] = 'Paul'
f_the = df3[df3.BookName == '1Thessalonians']
f_the['Author'] = 'Paul'
s_the = df3[df3.BookName == '2Thessalonians']
s_the['Author'] = 'Paul'
f_tim = df3[df3.BookName == '1Timothy']
f_tim['Author'] = 'Paul'
s_tim = df3[df3.BookName == '2Timothy']
s_tim['Author'] = 'Paul'
Titus = df3[df3.BookName == 'Titus']
Titus['Author'] = 'Paul'
Philemon = df3[df3.BookName == 'Philemon']
Philemon['Author'] = 'Paul'
Paul_letters = pd.concat([Romans,f_cor,s_cor,Gal,Eph,Phi,Col,f_the,s_the,f_tim,s_tim,Titus,Philemon],axis = 0)
Heb = df3[df3.BookName == 'Hebrews']
Heb['Author'] = 'Paul?'

In [61]:
g1 = cosine_text(Matthew)
g2 = cosine_text(Mark)
g3 = cosine_text(Luke)
g4 = cosine_text(John)
acts = cosine_text(Acts)
rms = cosine_text(Romans)
fcor = cosine_text(f_cor)
scor = cosine_text(s_cor)
gal = cosine_text(Gal)
eph = cosine_text(Eph)
phi = cosine_text(Phi)
col = cosine_text(Col)
fthe = cosine_text(f_the)
sthe = cosine_text(s_the)
ftim = cosine_text(f_tim)
stim = cosine_text(s_tim)
titus = cosine_text(Titus)
philemon = cosine_text(Philemon)
jams = cosine_text(James)
fpet = cosine_text(f_Peter)
spet = cosine_text(s_Peter)
fjohn = cosine_text(f_John)
sjohn = cosine_text(s_John)
tjohn = cosine_text(t_John.dropna())
jud = cosine_text(Jude)
rev = cosine_text(Rev)
hbr = cosine_text(Heb)

In [62]:
g11 = word_list(g1)
g21 = word_list(g2)
g31 = word_list(g3)
g41 = word_list(g4)
acts1 = word_list(acts)
rms1 = word_list(rms)
fcor1 = word_list(fcor)
scor1 = word_list(scor)
gal1 = word_list(gal)
eph1 = word_list(eph)
phi1 = word_list(phi)
col1 = word_list(col)
fthe1 = word_list(fthe)
sthe1 = word_list(sthe)
ftim1 = word_list(ftim)
stim1 = word_list(stim)
titus1 = word_list(titus)
philemon1 = word_list(philemon)
jams1 = word_list(jams)
fpet1 = word_list(fpet)
spet1 = word_list(spet)
fjohn1 = word_list(fjohn)
sjohn1 = word_list(sjohn)
tjohn1 = word_list(tjohn)
jud1 = word_list(jud)
rev1 = word_list(rev)
hbr1 = word_list(hbr)

In [63]:
documents = [g11,g21,g31,g41,acts1,rms1,fcor1,scor1,gal1,eph1,phi1,col1,fthe1,sthe1,ftim1,stim1,titus1,philemon1,jams1,fpet1,spet1,fjohn1,sjohn1,tjohn1,jud1,rev1]
s_dice = []
s_hel = []
s_jac = []
s_jen = []
for x in range(0,len(documents)):
    
    s_dice.append(dice(documents[x][0],documents[x][1]))
    s_hel.append(hellinger(documents[x][0],documents[x][1]))
    s_jac.append(jaccard(documents[x][0],documents[x][1]))
    s_jen.append(distance.jensenshannon(documents[x][0],documents[x][1]))

In [64]:
nlp = spacy.load('el_core_news_md')

In [65]:
s_cosine = []
documents = [g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,27))
doc1 = nlp(hbr)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine.append(similarity)

In [66]:
n_cosine = []
documents = [hbr,g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,27))
for x in range(1,len(documents)):
    two_books = [hbr,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m5 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Hebrews','Compared Book'])
    # df_m5.to_csv(str(bindex[x])+'compared Hebrews term weight matrix.csv', encoding = 'utf-16',sep = '\t', index=False)
    cos = tuple(cosine_similarity(df_m5)[0])[1]
    n_cosine.append(cos)

In [67]:
# dice, js divergence, hellinger distance
n_dice = []
n_hel = []
n_jac = []
n_jen = []
documents = [hbr,g1,g2,g3,g4,acts,rms,fcor,scor,gal,eph,phi,col,fthe,sthe,ftim,stim,titus,philemon,jams,fpet,spet,fjohn,sjohn,tjohn,jud,rev]
bindex = list(range(0,len(documents)))
for x in range(1,len(documents)):
    two_books = [hbr,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice.append(dice(m1[0],m2[0]))
    n_hel.append(hellinger(m1[0],m2[0]))
    n_jac.append(jaccard(m1[0],m2[0]))
    n_jen.append(distance.jensenshannon(m1[0],m2[0]))
    

In [68]:
Tyndale = {'SpacyCosine':s_cosine, 'SpacyDice':s_dice, 'SpacyHellinger':s_hel, 'SpacyJaccard':s_jac, 'SpacyJSdivergence':s_jen, 'NLTKCosine':n_cosine, 'NLTKDice':n_dice, 'NLTKHellinger':n_hel, 'NLTKJaccard':n_jac, 'NLTKJSdivergence':n_jen}
Tyndale = pd.DataFrame(Tyndale)

In [69]:
mat_s1 = cosine_text(Matthew)
mark_s1 = cosine_text(Mark)
luke_s1 = cosine_text(Luke_Book)
jam_s1 = cosine_text(James)
pet_s1 = cosine_text(Peter_letter)
john_s1 = cosine_text(John_Book.dropna())
jude_s1 = cosine_text(Jude)
pl_s1 = cosine_text(Paul_letters)

In [70]:
mat_s2 = word_list(mat_s1)
mark_s2 = word_list(mark_s1)
luke_s2 = word_list(luke_s1)
jam_s2 = word_list(jam_s1)
pet_s2 = word_list(pet_s1)
john_s2 = word_list(john_s1)
jude_s2 = word_list(jude_s1)
pl_s2 = word_list(pl_s1)

In [71]:
s_dice1 = []
s_hel1 = []
s_jac1 = []
s_jen1 = []
documents = [pl_s2,mat_s2,mark_s2,luke_s2,jam_s2,pet_s2,john_s2,jude_s2]
for x in range(0,len(documents)):
    s_dice1.append(dice(documents[x][0],documents[x][1]))
    s_hel1.append(hellinger(documents[x][0],documents[x][1]))
    s_jac1.append(jaccard(documents[x][0],documents[x][1]))
    s_jen1.append(distance.jensenshannon(documents[x][0],documents[x][1]))

In [72]:
s_cosine1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
doc1 = nlp(hbr)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine1.append(similarity)

In [73]:
n_cosine1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [hbr,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m6 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Paul','compared books'])
    # df_m6.to_csv(str(authorindex[x])+'author compare with Paul term weight matrix.csv', encoding = 'utf-8', index=False)
    cos = tuple(cosine_similarity(df_m6)[0])[1]
    n_cosine1.append(cos)

In [74]:
n_dice1 = []
n_hel1 = []
n_jac1 = []
n_jen1 = []
documents = [pl_s1,mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [hbr,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice1.append(dice(m1[0],m2[0]))
    n_hel1.append(hellinger(m1[0],m2[0]))
    n_jac1.append(jaccard(m1[0],m2[0]))
    n_jen1.append(distance.jensenshannon(m1[0],m2[0]))

In [75]:
Tyndale1 = {'SpacyCosine':s_cosine1, 'SpacyDice':s_dice1, 'SpacyHellinger':s_hel1, 'SpacyJaccard':s_jac1, 'SpacyJSdivergence':s_jen1, 'NLTKCosine':n_cosine1, 'NLTKDice':n_dice1, 'NLTKHellinger':n_hel1, 'NLTKJaccard':n_jac1, 'NLTKJSdivergence':n_jen1}
Tyndale1 = pd.DataFrame(Tyndale1)

In [76]:
mat_s4 = word_list1(mat_s1)
mark_s4 = word_list1(mark_s1)
luke_s4 = word_list1(luke_s1)
jam_s4 = word_list1(jam_s1)
pet_s4 = word_list1(pet_s1)
john_s4 = word_list1(john_s1)
jude_s4 = word_list1(jude_s1)

In [77]:
s_dice2 = []
s_hel2 = []
s_jac2 = []
s_jen2 = []
documents = [mat_s4,mark_s4,luke_s4,jam_s4,pet_s4,john_s4,jude_s4]
for x in range(0,len(documents)):
    s_dice2.append(dice(documents[x][0],documents[x][1]))
    s_hel2.append(hellinger(documents[x][0],documents[x][1]))
    s_jac2.append(jaccard(documents[x][0],documents[x][1]))
    s_jen2.append(distance.jensenshannon(documents[x][0],documents[x][1]))

In [78]:
s_cosine2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
doc1 = nlp(pl_s1)
for x in range(0,len(documents)):
    doc2 = nlp(documents[x])
    similarity = doc1.similarity(doc2)
    s_cosine2.append(similarity)

In [79]:
n_cosine2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [pl_s1,documents[x]]
    # Create the Document Term Matrix
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)

    # Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df_m7 = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['Paul','compared books'])
    # df_m6.to_csv(str(authorindex[x])+'author compare with Paul term weight matrix.csv', encoding = 'utf-8', index=False)
    cos = tuple(cosine_similarity(df_m7)[0])[1]
    n_cosine2.append(cos)

In [80]:
n_dice2 = []
n_hel2 = []
n_jac2 = []
n_jen2 = []
documents = [mat_s1,mark_s1,luke_s1,jam_s1,pet_s1,john_s1,jude_s1]
authorindex = list(range(0,len(documents)))
for x in range(0,len(documents)):
    two_books = [pl_s1,documents[x]]
    count_vectorizer = CountVectorizer(stop_words='greek')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(two_books)
    doc_term_matrix = sparse_matrix.todense()
    m1 = doc_term_matrix[0].tolist()
    m2 = doc_term_matrix[1].tolist()
    n_dice2.append(dice(m1[0],m2[0]))
    n_hel2.append(hellinger(m1[0],m2[0]))
    n_jac2.append(jaccard(m1[0],m2[0]))
    n_jen2.append(distance.jensenshannon(m1[0],m2[0]))

In [81]:
Tyndale2 = {'SpacyCosine':s_cosine2, 'SpacyDice':s_dice2, 'SpacyHellinger':s_hel2, 'SpacyJaccard':s_jac2, 'SpacyJSdivergence':s_jen2, 'NLTKCosine':n_cosine2, 'NLTKDice':n_dice2, 'NLTKHellinger':n_hel2, 'NLTKJaccard':n_jac2, 'NLTKJSdivergence':n_jen2}
Tyndale2 = pd.DataFrame(Tyndale2)

In [82]:
Tyndale_all = pd.concat([Tyndale, Tyndale1, Tyndale2], axis = 0)

In [85]:
all_index = pd.concat([NA28_all,SBL_all,Tyndale_all], axis = 1)

In [87]:
all_index.to_excel('SpaCy&NLTK_index.xlsx',index = False)